In [45]:
import overpy
import pandas as pd
import numpy as np
import folium


# Initialize Overpass API
api = overpy.Overpass()

# Write Overpass query
query = """
[out:json];
area["ISO3166-1"="CH"][admin_level=2];
(node["amenity"="fuel"](area);
 way["amenity"="fuel"](area);
 rel["amenity"="fuel"](area);
);
out center;
"""

# Run the query
result = api.query(query)

# Prepare a list to store the data
data = []

# Loop over the result
for element in result.nodes + result.ways + result.relations:
    # Get the operator tag, if it exists
    operator = element.tags.get('operator', None)
    # Append the data to the list
    data.append({'id': element.id, 'operator': operator})

# Convert the list to a DataFrame
df = pd.DataFrame(data)

df.to_csv('fuel_stations.csv', sep=';')
df.to_json('fuel_stations.json')


sorted_df = df.pivot_table(index=['operator'], values=['id'], aggfunc='count').sort_values(by='id', ascending=False)

print(sorted_df.head())

                  id
operator            
Migrol           157
BP                70
Coop Mineraloel   68
Schweizer Armee   64
Avia              60


In [48]:
import folium

# Read the spatial data
polys = 'fuel_stations.json'

# Initialize the map
m = folium.Map(location=[47.44, 8.65], zoom_start=10)

# Add markers for each petrol station
for index, row in df.iterrows():
    operator = row['operator']
    lat = result.nodes[index].lat
    lon = result.nodes[index].lon
    popup_text = f"Brand: {operator}"
    folium.Marker([lat, lon], popup=popup_text).add_to(m)

# Display the map
m


# Subset of gas stations by brand
locations = df[["lat", "lon", "tags.name"]]
print(locations.head(5))

# Create map
map = folium.Map(location=[locations.lat.mean(), 
                           locations.lon.mean()], 
                 zoom_start=14, 
                 control_scale=True)

# Add maker symbols
for index, location_info in locations.iterrows():
    folium.Marker([location_info["lat"], 
                   location_info["lon"]], 
                  popup=location_info["tags.name"]).add_to(map)
